https://cloud.tencent.com/developer/article/1169271
### 无监督学习
我的目标是根据嫌疑人的表现的相似性来聚集案件。如果可以创建行为概况，那么我们就可以根据它们属于哪个集群，更有效地给嫌疑犯分配各种外展活动。

### 1. PCA数据转换
有39个特征描述了我们可以跟踪的嫌疑犯的行为。这些包括“受损”，“推动”和“投掷物品”等动作。 唯一的问题是，这些是二进制特征，处理它们可能有点棘手。\
\
经过多次失败的系统聚类实验以及使用不同相异性度量的测试后，我发现在使用主成分分析法对变量进行转换之后应用K均值聚类产生了解释度非常高的聚类。\
整个过程可以总结在下图中：
![title](img/1x1wks09v5.png)
通常情况下，我们认为通过PCA选择解释足够差异的主要成分的数量，是降低维度的一种手段。 \
但是，我们的目标不是要减少我们拥有的特征量，我们的目标是将它们转换为我们可以聚类的数字数据。为此，我们将所有39个主成分得分（又称特征向量），保留100％的原始方差并对它们进行聚类。
### 2.聚类
K均值聚类中的目标函数是最小化群内差异。 看看陡坡图，5、6个集群看起来是正确的。 在对两者进行试验之后，我得出结论认为5个集群最具可解释性。
### 3.集群档案
在为每个观察分配一个聚类标签并将它们与原始数据集匹配后，描述它们就变得非常容易。 回顾二进制数据，我们可以简单地将每个特性和每个集群中的所有1加起来。 具有较高总和的特征在描述该集群时将被赋予更多权重。下面是集群前3个最突出的特征的例子，这些特征被描述为“在影响下相遇”：
![title](img/oghjcaqfds.png)
根据上面的示例分析，最终集群看起来像这样！ 每个案件将被标记为属于这些集群中的一个，并且每种情况下的嫌疑人都被假定为采用其中一个档案。
![title](img/lzonbbnocr.jpeg)


### 监督学习

在开始任何监督学习方法之前，我首先需要一个可以帮助指导机器学习的功能。重申一下，我们的目标是能够对案件进行分类，不管它是否亲密，因为发现并非所有案件都被正确分类。

#### 1. 特征工程—标记“亲密”
#### 2. 选择一个模型
现在我已经对每个案例做了标记（“亲密”/“非亲密”），我需要建立一个分类模型。出于多种原因，我决定采用Logistic回归模型，对多项式朴素贝叶斯（Multinomial Naive Bayes）分类器进行建模：

* 由于训练规模较小，生成的朴素贝叶斯（Naive 	Bayes）模型将超越本文Andrew 	Ng所述的判别性Logistic回归模型；
* 朴素贝叶斯（Naive 	Bayes）及其独立特征的假设使得模型更简单，更普通，因此变体更少；
* 朴素贝叶斯（Naive 	Bayes）因其用于文本数据和垃圾邮件检测而名声鹤唳。
增加最后一点非常有趣的，提供的数据集包含了现场警官的文字叙述。 我们可以通过警官叙述的词语选择来检测亲密关系吗？我决定找出答案。

#### 3. 自然语言处理
为了运行朴素贝叶斯分类器（Multinomial Naive Bayes classifier），我需要首先清理文本数据。 在Python中使用NLTK模块，采取了以下步骤：
* 标记叙述（使用RegexpTokenizer）
* 删除无用词（使用无用词）
* 应用词形（使用WordNetLemmatizer）
处理完文本之后，我将标记化和词元化的叙述转换为一个交易对象，其中每个单词都是自己的特征，每一行都是叙述或“文档”。 这也可以被称为“文档术语矩阵”。

#### 4. 降维
文档术语矩阵生成了3,805个特征。 为了减少这一点，以提高我们的模型的准确性，不够频繁的特征被删除。通过使用验证集合，我得出结论，删除所有出现少于两次的单词就足够了。这已经将维度降低到1861个特征。

#### 5. 调整多项式朴素贝叶斯（Multinomial Naive Bayes）
多项式朴素贝叶斯分类器（Multinomial Naive Bayes classifier）默认将alpha设置为1. Alpha是一个平滑参数，用于处理出现在未在训练集中训练的保留集中的单词。 使用GridSearchCV，alpha被调整为2.53。

#### 6. 模型评估
在用α= 2.53重置多项式朴素贝叶斯（Multinomial Naive Bayes）并将数据分解成训练集和测试集后，结果出人意料地好。

训练精度：84％

测试精度：81％

#### 7. 结论
总之，该模型的真阳性，即敏感性为80.4％（82 /（82 + 20）），该模型的真阴性又称特异性为82.4％（14 /（3 + 14））。两者都远超我的预期，由于叙述为许多不同的警官所写，而每个警官都有自己的写作风格。但是模型显示，无论警官是谁，处理亲密案件时，都会倾向于频繁地使用某些关键词。这才是该模型性能的关键。

#### 尾语
希望上述模型能够被我们的社区广泛使用。聚类可以非常有帮助，特别是当我们没有足够的数据为我们的案例提供标签时。它可以让我们快速了解嫌疑人是什么样的（实际上我更愿意称他们为病人）。 鉴于我们确实有足够数据的幸运情况，朴素贝叶斯分类器可以大大减少将潜在患者筛选到治疗计划所需的时间。早期发现早期干预可以减少我们社区中的暴力案件，这只是数据科学可以做的许多有价值的事情之一。迫不及待想要看看还有什么事情可以做！